In [1]:
from kafka import KafkaConsumer
from kafka import TopicPartition
from datetime import datetime
import pandas as pd
from cassandra.cluster import Cluster
import json
from botocore.vendored import requests
import findspark
findspark.init()
import pyspark.sql 
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.ml.classification import LogisticRegression
import pandas as pd
from cassandra.cluster import Cluster
import numpy as np
from pyspark.sql.types import *

In [2]:
# #------------------------------- SPARK CONTEXT -------------------------------
sc = SparkContext('local')
spark = SparkSession(sc)
spark = SparkSession.builder.appName('CC_Fraud').getOrCreate()


In [3]:
#------------------------------- Load Model from File System -------------------------------

def MachineLearningModelLoading():
	path = r"C:\Users\saritajoshi\Documents\Repos\Credit-Card-Fraud-Detection-Spark\model"
	newmodel = PipelineModel.load(path)
	return newmodel
	
newmodel = MachineLearningModelLoading()


In [4]:
# feature_columns = [col for col in df.columns if col != "is_fraud"]
# assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")

In [5]:
#------------------------------------ Data Cleaning -------------------------------------
def predicting_status(data_all,newmodel):
	select_cols = ['amt', 'lat', 'long', 'city_pop', 'unix_time','merch_lat', 'merch_long','merchant_label','category_label', 'gender_label', 'job_label'] #all numeric
	print(data_all.columns)
	data = data_all.select(select_cols)
	pred1 = newmodel.transform(data)
	return pred1.select("prediction")

# df = spark.read.json(sc.parallelize([dd]))
# res = predicting_status(df, newmodel)

In [6]:

#------------------------------------------- CASSANDRA CONNECTION -------------------------------------------------------
cluster = Cluster()
session = cluster.connect()

session.execute("CREATE KEYSPACE IF NOT EXISTS bigdata WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3}")
session.set_keyspace("bigdata")

#session.execute("DROP TABLE testing")
# session.execute("CREATE TABLE IF NOT EXISTS testing (txn_id varchar PRIMARY KEY,cc_no bigint ,txn_time varchar, year smallint, month smallint, day smallint, hour smallint, minute smallint,amount double, cc_provider int, merchant varchar,merchant_index smallint, country_name varchar, country_code varchar,country_index smallint, age int, marital_status varchar,marital_status_index smallint,gender varchar,gender_index smallint,loan varchar,loan_index smallint, status varchar,status_index smallint)")


Traceback (most recent call last):
  File "cassandra\cluster.py", line 3577, in cassandra.cluster.ControlConnection._reconnect_internal
  File "cassandra\cluster.py", line 3599, in cassandra.cluster.ControlConnection._try_connect
  File "cassandra\cluster.py", line 1670, in cassandra.cluster.Cluster.connection_factory
  File "cassandra\connection.py", line 846, in cassandra.connection.Connection.factory
  File "c:\Users\saritajoshi\Documents\Repos\Credit-Card-Fraud-Detection-Spark\venv\Lib\site-packages\cassandra\io\asyncorereactor.py", line 354, in __init__
    self._connect_socket()
  File "cassandra\connection.py", line 951, in cassandra.connection.Connection._connect_socket
ConnectionRefusedError: [Errno 10061] Tried connecting to [('127.0.0.1', 9042)]. Last error: No connection could be made because the target machine actively refused it


In [ ]:
topic_name = 'transaction_data'
keyspace = 'bigdata'
table = 'transaction_data'

In [54]:
consumer = KafkaConsumer(topic_name, group_id=None, bootstrap_servers=['localhost:9092'])
for msg in consumer:
	# print('msg', msg)
	msg=msg.value.decode("utf-8")
	output = json.loads(msg)
	df = spark.read.json(sc.parallelize([msg]))
	status = predicting_status(df,newmodel)
	# print(status)
	status = int(status.collect()[0][0])
	output['is_fraud_prediction'] = status
	# print('processed...', status)

	insert_query = f"INSERT INTO {keyspace}.{table} JSON '{json.dumps(output)}';"
	session.execute(insert_query)

['amt', 'category', 'category_label', 'cc_num', 'city', 'city_pop', 'dob', 'first', 'gender', 'gender_label', 'is_fraud', 'job', 'job_label', 'last', 'lat', 'long', 'merch_lat', 'merch_long', 'merchant', 'merchant_label', 'state', 'street', 'trans_date_trans_time', 'trans_num', 'unix_time', 'zip']
['amt', 'category', 'category_label', 'cc_num', 'city', 'city_pop', 'dob', 'first', 'gender', 'gender_label', 'is_fraud', 'job', 'job_label', 'last', 'lat', 'long', 'merch_lat', 'merch_long', 'merchant', 'merchant_label', 'state', 'street', 'trans_date_trans_time', 'trans_num', 'unix_time', 'zip']
['amt', 'category', 'category_label', 'cc_num', 'city', 'city_pop', 'dob', 'first', 'gender', 'gender_label', 'is_fraud', 'job', 'job_label', 'last', 'lat', 'long', 'merch_lat', 'merch_long', 'merchant', 'merchant_label', 'state', 'street', 'trans_date_trans_time', 'trans_num', 'unix_time', 'zip']
['amt', 'category', 'category_label', 'cc_num', 'city', 'city_pop', 'dob', 'first', 'gender', 'gender_l

SyntaxException: <Error from server: code=2000 [Syntax error in CQL query] message="line 1:414 mismatched input 's' expecting EOF (... INTO bigdata.transaction_data JSON '{"trans_date_trans_time": "2020-08-07 12:17:48", "cc_num": 4400011257587661852, "merchant": "fraud_Monahan, Hermann and Johns", "category": "travel", "amt": 8.84, "first": "Marissa", "last": "Powell", "gender": "F", "street": "474 Allen Haven", "city": "North Loup", "state": "NE", "zip": 68859, "lat": 41.4972, "long": -98.7858, "city_pop": 509, "job": "Nurse, children'[s]...)">

In [ ]:

consumer.close()
spark.stop()

In [47]:
insert_query = f"INSERT INTO {keyspace}.{table} JSON '{json.dumps(output)}';"
session.execute(insert_query)

In [43]:
output['is_fraud_prediction'] = 0